# **Chapter 10: Modern Data Architectures in the Cloud**

## Introduction: The Data-First Imperative

As organizations migrate workloads to the cloud, the volume, velocity, and variety of data have exploded exponentially. While Chapter 9 explored how serverless architectures handle compute elasticity and event processing, modern applications are fundamentally data-driven. Whether it's real-time personalization, predictive analytics, or AI-powered insights, the ability to ingest, store, process, and analyze data at scale separates innovative organizations from their competitors.

Traditional on-premises data architectures struggled with rigid schemas, capacity planning challenges, and the inability to handle unstructured data. Cloud-native data architectures eliminate these constraints through decoupled storage and compute, serverless query engines, and unified analytics platforms that can process petabytes of data without provisioning a single server.

This chapter examines the evolution from monolithic databases to modern data platforms, covering data lakes, warehouses, streaming pipelines, and the integration of machine learning workflows. We will explore how these components work synergistically with the serverless patterns from Chapter 9 to create intelligent, data-rich applications.

---

## 10.1 Data Lakes vs. Data Warehouses: Architecture Patterns

The choice between data lakes and data warehouses represents fundamental architectural decisions about schema enforcement, data structure, and analytical workloads. Modern organizations often implement both in a unified "lakehouse" architecture.

### 10.1.1 Understanding the Data Lake

A data lake is a centralized repository that allows you to store all your structured and unstructured data at any scale. Unlike traditional databases, data lakes store data in its raw format without requiring upfront schema definition (schema-on-read).

**Key Characteristics:**
- **Raw Storage:** Stores data in native formats (JSON, CSV, Parquet, images, videos, logs)
- **Schema-on-Read:** Structure is applied when data is accessed, not when ingested
- **Infinite Scale:** Object storage (S3, Azure Data Lake Storage, GCS) provides virtually unlimited capacity
- **Cost-Effective:** Store everything; cheap storage for exploratory analytics and data science

**The Modern Data Lake Architecture (Three-Layer Pattern):**

Industry best practices organize data lakes into distinct zones to balance agility with governance:

1. **Bronze Layer (Raw):** Data lands here exactly as received from source systems. Immutable, append-only storage of historical data. No transformations applied.

2. **Silver Layer (Cleansed):** Data quality rules applied, deduplication, type casting, and basic normalization. Ready for analytical querying but not yet business-aggregated.

3. **Gold Layer (Curated):** Business-level aggregates, star schemas, and feature engineering for ML models. Optimized for specific consumption patterns.

**Terraform Implementation of a Secure Data Lake:**

```hcl
# Infrastructure for a three-tier data lake on AWS S3
resource "aws_s3_bucket" "data_lake" {
  bucket = "company-data-lake-${var.environment}"
  
  tags = {
    Environment = var.environment
    Layer       = "multi-zone"
  }
}

# Bronze Zone - Raw data ingestion
resource "aws_s3_bucket" "bronze" {
  bucket = "${aws_s3_bucket.data_lake.bucket}-bronze"
}

resource "aws_s3_bucket_lifecycle_configuration" "bronze_lifecycle" {
  bucket = aws_s3_bucket.bronze.id
  
  rule {
    id     = "transition-to-glacier"
    status = "Enabled"
    
    transition {
      days          = 90
      storage_class = "GLACIER_IR"  # Instant Retrieval for occasional access
    }
    
    noncurrent_version_expiration {
      noncurrent_days = 30
    }
  }
}

# Silver Zone - Cleaned and validated data
resource "aws_s3_bucket" "silver" {
  bucket = "${aws_s3_bucket.data_lake.bucket}-silver"
}

resource "aws_s3_bucket_versioning" "silver_versioning" {
  bucket = aws_s3_bucket.silver.id
  versioning_configuration {
    status = "Enabled"
  }
}

# Gold Zone - Business-ready aggregates
resource "aws_s3_bucket" "gold" {
  bucket = "${aws_s3_bucket.data_lake.bucket}-gold"
}

# Security: Bucket policies to enforce encryption and access logging
resource "aws_s3_bucket_server_side_encryption_configuration" "encryption" {
  for_each = toset([aws_s3_bucket.bronze.id, aws_s3_bucket.silver.id, aws_s3_bucket.gold.id])
  
  bucket = each.value
  
  rule {
    apply_server_side_encryption_by_default {
      sse_algorithm     = "aws:kms"
      kms_master_key_id = aws_kms_key.data_lake_key.arn
    }
    bucket_key_enabled = true
  }
}

# Cross-region replication for disaster recovery
resource "aws_s3_bucket_replication_configuration" "replication" {
  role   = aws_iam_role.replication.arn
  bucket = aws_s3_bucket.gold.id
  
  rule {
    id     = "replicate-gold-tier"
    status = "Enabled"
    
    destination {
      bucket        = aws_s3_bucket.gold_dr.arn
      storage_class = "STANDARD"
      
      encryption_configuration {
        replica_kms_key_id = aws_kms_key.data_lake_key_dr.arn
      }
    }
    
    source_selection_criteria {
      sse_kms_encrypted_objects {
        status = "Enabled"
      }
    }
  }
}
```

**Explanation:**
- **Zone Separation:** Physical separation via buckets enforces data quality boundaries and allows different lifecycle policies (e.g., raw data moves to Glacier after 90 days, curated data remains immediately accessible)
- **Encryption:** All tiers use AWS KMS for server-side encryption; bucket keys reduce KMS API costs for high-throughput workloads
- **Versioning:** Enabled on Silver and Gold tiers to protect against accidental deletion and support time-travel queries
- **Replication:** Only the Gold tier replicates cross-region, as it contains business-critical aggregated data, while raw data can be re-ingested if needed

### 10.1.2 The Data Warehouse: Structured Analytics

While data lakes excel at storing vast amounts of diverse data, data warehouses optimize for high-performance SQL analytics on structured data. They employ columnar storage, massively parallel processing (MPP), and sophisticated query optimizers.

**Key Characteristics:**
- **Schema-on-Write:** Data must conform to defined schemas before ingestion
- **Columnar Storage:** Optimized for aggregations (SUM, AVG, COUNT) across billions of rows
- **SQL-Native:** Standard SQL interface for business intelligence tools
- **Strong Consistency:** ACID transactions for accurate reporting

**Modern Cloud Data Warehouses:**
- **Amazon Redshift:** Petabyte-scale warehouse with RA3 nodes (managed storage), Spectrum (query S3 directly), and Serverless option
- **Google BigQuery:** Fully serverless, separates compute from storage, charges per query (on-demand) or per slot (reserved)
- **Azure Synapse Analytics:** Unified analytics platform combining SQL pools, Spark pools, and data exploration
- **Snowflake:** Cross-cloud warehouse with instant elasticity, zero-copy cloning, and time travel

**BigQuery Example: Creating a Partitioned and Clustered Table:**

Partitioning and clustering are critical optimization techniques that reduce query costs and improve performance by limiting the data scanned.

```sql
-- Creating an optimized table for time-series analytics
CREATE OR REPLACE TABLE `analytics.orders_processed`
(
    order_id STRING NOT NULL,
    customer_id STRING NOT NULL,
    order_timestamp TIMESTAMP NOT NULL,
    amount NUMERIC,
    currency STRING,
    status STRING,
    product_category STRING,
    device_type STRING,
    geo_region STRING
)
PARTITION BY DATE(order_timestamp)  -- Daily partitions
CLUSTER BY geo_region, product_category  -- Column order matters for filtering
OPTIONS(
    description = "Processed orders with optimization for regional and category analysis",
    labels = [("team", "analytics"), ("sensitivity", "confidential")]
);

-- Query that leverages partitioning and clustering
-- This query only scans the last 7 days of data and specific regions
SELECT 
    product_category,
    COUNT(*) as order_count,
    SUM(amount) as total_revenue
FROM `analytics.orders_processed`
WHERE order_timestamp >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY)
  AND geo_region IN ('US-West', 'US-East')
  AND product_category = 'Electronics'
GROUP BY product_category;

-- Best Practice: Use preview to estimate bytes processed
-- BigQuery shows "This query will process 12.3 MB" before execution
```

**Why This Matters:**
- **Partitioning:** Divides the table into daily segments. A query for one day scans only that partition, not the entire table (potentially saving 99% of costs for multi-year datasets)
- **Clustering:** Organizes data within partitions by column values. When filtering by `geo_region` and `product_category`, BigQuery skips blocks that don't contain matching values
- **Cost Control:** BigQuery charges by bytes scanned ($5 per TB). Proper optimization can reduce costs from hundreds of dollars to cents per query

### 10.1.3 The Lakehouse Architecture: Best of Both Worlds

Modern architectures blur the lines between lakes and warehouses through the "Lakehouse" pattern, enabled by open table formats like Apache Iceberg, Delta Lake, and Apache Hudi.

**Benefits:**
- **ACID transactions** on data lake storage (S3/ADLS)
- **Time travel:** Query data as it existed at any point in time
- **Schema evolution:** Add columns without breaking existing queries
- **Unified governance:** Single security model across all data

**Delta Lake Example (Azure Databricks/AWS Glue):**

```python
from delta import DeltaTable, configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, current_timestamp

# Initialize Spark with Delta Lake
builder = SparkSession.builder \
    .appName("LakehouseETL") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

# Bronze ingestion: Append raw JSON data
raw_df = spark.read.json("s3a://data-lake-bronze/events/")

# Write to Delta table (Silver tier) with merge schema evolution
(raw_df.write
    .format("delta")
    .mode("append")
    .option("mergeSchema", "true")  # Automatically handle new columns
    .save("s3a://data-lake-silver/events/"))

# Create Silver table with quality checks
silver_df = spark.read.format("delta").load("s3a://data-lake-silver/events/")

# Data quality validation and cleansing
clean_df = (silver_df
    .filter(col("user_id").isNotNull())
    .filter(col("event_timestamp") > "2020-01-01")
    .withColumn("processed_at", current_timestamp()))

# Upsert (Merge) into curated Gold table
# This handles late-arriving data and duplicates idempotently
delta_table = DeltaTable.forPath(spark, "s3a://data-lake-gold/user-events/")

(delta_table.alias("target")
    .merge(clean_df.alias("source"), "target.event_id = source.event_id")
    .whenMatchedUpdateAll()
    .whenNotMatchedInsertAll()
    .execute())

# Time Travel: Query data as it was yesterday
yesterday_df = (spark.read
    .format("delta")
    .option("timestampAsOf", "2026-02-01T00:00:00Z")
    .load("s3a://data-lake-gold/user-events/"))

# Optimize table layout for query performance
spark.sql("OPTIMIZE delta.`s3a://data-lake-gold/user-events/` ZORDER BY (user_id)")
```

**Key Operations Explained:**
- **Merge/Upsert:** Atomically updates existing records and inserts new ones, handling late-arriving data without duplicates
- **Time Travel:** Audit compliance and debugging by querying historical states without maintaining separate snapshots
- **Z-Ordering:** Co-locates related data in the same files, reducing I/O for analytical queries that filter on specific columns

---

## 10.2 Data Pipelines: ETL vs. ELT Patterns

Data pipelines move and transform data between systems. Cloud architectures have shifted from traditional ETL (Extract, Transform, Load) to ELT (Extract, Load, Transform) to leverage the scalability of target warehouses.

### 10.2.1 Understanding ETL vs. ELT

**ETL (Extract, Transform, Load):**
- Transformations occur in a dedicated engine (often Spark or Dataflow) before loading
- **Pros:** Data is clean before reaching the warehouse; protects production systems from heavy transformation loads
- **Cons:** Requires capacity planning for transformation clusters; schema changes require pipeline modifications

**ELT (Extract, Load, Transform):**
- Raw data loads directly into the warehouse; transformations occur there using SQL
- **Pros:** Leverages warehouse scalability; analysts can transform data using familiar SQL; schema-on-read flexibility
- **Cons:** Requires disciplined cost management; raw data quality issues can propagate

**Modern Approach:** Hybrid ELT/ETL where lightweight cleansing happens during extraction (ETL), but heavy aggregations occur in the warehouse (ELT).

### 10.2.2 Managed Pipeline Services

Cloud providers offer fully managed, serverless ETL/ELT services that eliminate infrastructure management:

**AWS Glue (Serverless Spark):**
- Automatically generates ETL code from data sources
- Glue Data Catalog serves as the central metadata repository
- Glue Studio provides visual ETL development

**Azure Data Factory:**
- Cloud-based ETL and data integration service
- Orchestrates data movement and transformation across cloud and on-premises sources
- Integration with Azure Synapse Analytics for big data processing

**Google Cloud Dataflow:**
- Apache Beam-based streaming and batch processing
- Autoscaling from zero to thousands of workers
- Exactly-once processing semantics for streaming data

**AWS Glue ETL Job Example (Python/PySpark):**

```python
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
from pyspark.sql.functions import udf, col, to_timestamp
from pyspark.sql.types import StringType

# Initialize Glue context
args = getResolvedOptions(sys.argv, ['JOB_NAME'])
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# Read from Data Catalog (Bronze layer)
datasource = glueContext.create_dynamic_frame.from_catalog(
    database = "ecommerce_raw",
    table_name = "orders_json",
    transformation_ctx = "datasource"
)

# Convert to DataFrame for complex transformations
df = datasource.toDF()

# Data quality transformations
@udf(StringType())
def normalize_phone(phone):
    """Standardize phone number formats"""
    if not phone:
        return None
    digits = ''.join(filter(str.isdigit, phone))
    return f"+1-{digits[:3]}-{digits[3:6]}-{digits[6:]}" if len(digits) == 10 else None

# Apply transformations
cleaned_df = (df
    .filter(col("order_id").isNotNull())  # Remove null keys
    .withColumn("order_date", to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss"))
    .withColumn("phone_normalized", normalize_phone(col("customer_phone")))
    .dropDuplicates(["order_id"])  # Idempotent processing
    .cache()  # Optimize for multiple sinks
)

# Convert back to DynamicFrame for Glue features
cleaned_dynamic_frame = DynamicFrame.fromDF(cleaned_df, glueContext, "cleaned")

# Write to Silver layer in Parquet format (columnar, compressed)
glueContext.write_dynamic_frame.from_options(
    frame = cleaned_dynamic_frame,
    connection_type = "s3",
    connection_options = {
        "path": "s3://data-lake-silver/orders/",
        "partitionKeys": ["year", "month", "day"]  # Hive-style partitioning
    },
    format = "parquet",
    transformation_ctx = "datasink"
)

# Also write to Redshift (Gold layer) for immediate BI access
glueContext.write_dynamic_frame.from_jdbc_conf(
    frame = cleaned_dynamic_frame,
    catalog_connection = "redshift-connection",
    connection_options = {
        "dbtable": "public.orders_staging",
        "database": "analytics"
    },
    redshift_tmp_dir = "s3://temp-bucket/redshift/",
    transformation_ctx = "redshift_sink"
)

job.commit()
```

**Architecture Highlights:**
- **DynamicFrames:** Glue-specific abstraction that handles schema variations gracefully (missing fields, type changes)
- **Pushdown Predicates:** When reading from JDBC sources, Glue pushes filters to the database to minimize data transfer
- **Bookmarking:** Glue automatically tracks processed data to enable incremental loads without manual offset management
- **Parquet Format:** Columnar storage with Snappy compression provides 10x better query performance than JSON

### 10.2.3 Orchestration with Apache Airflow/MWAA

Complex pipelines require orchestration to handle dependencies, retries, and scheduling. Apache Airflow (available as Amazon MWAA, Google Cloud Composer, or Azure Managed Airflow) is the industry standard.

**DAG (Directed Acyclic Graph) Definition:**

```python
from airflow import DAG
from airflow.providers.amazon.aws.operators.glue import GlueJobOperator
from airflow.providers.amazon.aws.operators.redshift_data import RedshiftDataOperator
from airflow.providers.amazon.aws.sensors.glue import GlueJobSensor
from airflow.utils.task_group import TaskGroup
from datetime import datetime, timedelta

default_args = {
    'owner': 'data-engineering',
    'depends_on_past': False,
    'email_on_failure': True,
    'email': ['data-team@company.com'],
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    'sla': timedelta(hours=2)  # Service Level Agreement
}

with DAG(
    'ecommerce_daily_etl',
    default_args=default_args,
    description='Daily ETL from raw data to Redshift',
    schedule_interval='0 3 * * *',  # Daily at 3 AM
    start_date=datetime(2026, 1, 1),
    catchup=False,
    tags=['ecommerce', 'daily', 'critical'],
    max_active_runs=1  # Prevent overlapping executions
) as dag:

    # Task Group for Bronze ingestion (parallelizable)
    with TaskGroup(group_id='bronze_ingestion') as bronze_group:
        ingest_orders = GlueJobOperator(
            task_id='ingest_orders',
            job_name='bronze_orders_ingestion',
            region_name='us-east-1',
            wait_for_completion=False  # Async execution
        )
        
        ingest_customers = GlueJobOperator(
            task_id='ingest_customers',
            job_name='bronze_customers_ingestion',
            region_name='us-east-1',
            wait_for_completion=False
        )
        
        ingest_products = GlueJobOperator(
            task_id='ingest_products',
            job_name='bronze_products_ingestion',
            region_name='us-east-1',
            wait_for_completion=False
        )
    
    # Wait for all bronze jobs
    wait_for_bronze = GlueJobSensor(
        task_id='wait_for_bronze',
        job_name='dummy',  # Updated dynamically via XCom
        region_name='us-east-1',
        mode='reschedule',  # Free up worker slot while waiting
        poke_interval=60
    )
    
    # Silver transformation (single job for consistency)
    silver_transformation = GlueJobOperator(
        task_id='silver_transform',
        job_name='silver_orders_transformation',
        region_name='us-east-1',
        script_args={
            '--TARGET_DATE': '{{ ds }}',  # Jinja templating for execution date
            '--ENV': 'production'
        }
    )
    
    # Gold layer: Redshift transformations using SQL
    gold_aggregations = RedshiftDataOperator(
        task_id='update_daily_metrics',
        cluster_identifier='analytics-cluster',
        database='analytics',
        sql="""
            BEGIN;
            
            -- Upsert daily aggregates
            DELETE FROM daily_metrics 
            WHERE metric_date = '{{ ds }}';
            
            INSERT INTO daily_metrics
            SELECT 
                DATE(order_date) as metric_date,
                COUNT(*) as total_orders,
                SUM(amount) as total_revenue,
                COUNT(DISTINCT customer_id) as unique_customers
            FROM silver.orders
            WHERE DATE(order_date) = '{{ ds }}'
            GROUP BY 1;
            
            COMMIT;
        """,
        aws_conn_id='aws_default'
    )
    
    # Data quality check
    quality_check = RedshiftDataOperator(
        task_id='check_data_quality',
        cluster_identifier='analytics-cluster',
        database='analytics',
        sql="""
            SELECT CASE 
                WHEN COUNT(*) > 1000 AND SUM(amount) > 0 THEN 'PASS'
                ELSE 'FAIL'
            END as quality_check
            FROM daily_metrics
            WHERE metric_date = '{{ ds }}'
        """
    )
    
    # Define dependencies
    bronze_group >> wait_for_bronze >> silver_transformation >> gold_aggregations >> quality_check
```

**Orchestration Best Practices:**
- **Task Groups:** Group related tasks for visual organization and parallel execution
- **Sensors vs. Operators:** Use Sensors (like `GlueJobSensor`) to wait for external resources without consuming worker slots (mode='reschedule')
- **SLAs:** Define service level agreements to detect late pipelines
- **Backfilling:** Airflow's `catchup=False` prevents automatic backfills, but manual backfills can be triggered for historical data processing

---

## 10.3 Streaming Data and Real-Time Analytics

Batch processing (running jobs every hour or day) is insufficient for use cases requiring immediate action: fraud detection, IoT monitoring, real-time personalization, and operational dashboards. Streaming architectures process data continuously as it arrives.

### 10.3.1 Streaming Fundamentals

**Stream Processing Concepts:**
- **Event Time vs. Processing Time:** Event time is when the event occurred; processing time is when the system handles it. Watermarks handle late-arriving data.
- **Windowing:** Aggregating events over time windows (tumbling, sliding, session windows)
- **Exactly-Once Semantics:** Ensuring each event affects results exactly once, even during failures

**Managed Streaming Services:**
- **Amazon Kinesis:** Managed Kafka alternative with Data Streams (provisioning) and Data Firehose (auto-scaling delivery to S3/Redshift/OpenSearch)
- **Azure Event Hubs:** Kafka-compatible event streaming platform with Capture feature (automatic archiving to Blob Storage)
- **Google Pub/Sub:** Global messaging service with at-least-once delivery and Push/Pull subscription models
- **Confluent Cloud:** Fully managed Apache Kafka across clouds

### 10.3.2 Real-Time Stream Processing

**Apache Flink (Amazon Managed Flink, Azure Stream Analytics):**
Stateful stream processing with sub-second latency, handling complex event processing (CEP) and windowed aggregations.

**Kinesis Data Analytics (Flink) Example:**

```java
// Flink job for real-time fraud detection
public class FraudDetectionJob {
    
    public static void main(String[] args) throws Exception {
        StreamExecutionEnvironment env = StreamExecutionEnvironment.getExecutionEnvironment();
        
        // Configure checkpointing for exactly-once semantics
        env.enableCheckpointing(5000); // 5-second checkpoints
        env.getCheckpointConfig().setCheckpointingMode(CheckpointingMode.EXACTLY_ONCE);
        
        // Read from Kinesis Data Stream
        FlinkKinesisConsumer<Transaction> source = new FlinkKinesisConsumer<>(
            "transactions",
            new TransactionSchema(),
            kinesisConsumerConfig()
        );
        
        DataStream<Transaction> transactions = env.addSource(source);
        
        // Key by customer and detect patterns within 10-minute windows
        Pattern<Transaction, ?> fraudPattern = Pattern
            .<Transaction>begin("start")
            .where(new SimpleCondition<Transaction>() {
                public boolean filter(Transaction t) {
                    return t.amount > 10000; // Large transaction
                }
            })
            .next("middle")
            .where(new SimpleCondition<Transaction>() {
                public boolean filter(Transaction t) {
                    return t.location != t.previousLocation; // Different location
                }
            })
            .within(Time.minutes(10));
        
        // Apply CEP pattern
        PatternStream<Transaction> patternStream = CEP.pattern(
            transactions.keyBy(Transaction::getCustomerId),
            fraudPattern
        );
        
        // Alert generation
        DataStream<Alert> alerts = patternStream
            .select(new PatternSelectFunction<Transaction, Alert>() {
                public Alert select(Map<String, List<Transaction>> pattern) {
                    Transaction start = pattern.get("start").get(0);
                    Transaction middle = pattern.get("middle").get(0);
                    return new Alert(start.customerId, "SUSPICIOUS_ACTIVITY", 
                        String.format("Large tx followed by location change: %s to %s", 
                        start.location, middle.location));
                }
            });
        
        // Sink to SNS for notifications
        alerts.addSink(new SNSAlertSink());
        
        env.execute("Fraud Detection Pipeline");
    }
}
```

### 10.3.3 Lambda Architecture vs. Kappa Architecture

**Lambda Architecture (Traditional):**
- **Batch Layer:** Process all historical data (accurate but high latency)
- **Speed Layer:** Process real-time data (approximate, low latency)
- **Serving Layer:** Merge batch and speed views for querying
- **Cons:** Complex, requires maintaining two codebases (batch and stream)

**Kappa Architecture (Modern):**
- **Unified Processing:** Use streaming for both real-time and historical reprocessing
- **Reprocessing:** Replay streams from beginning to rebuild state
- **Pros:** Single codebase, simplified architecture
- **Tools:** Kafka with Kafka Streams, Kinesis with Lambda/Flink

**Implementing Kappa Architecture with DynamoDB Streams and Lambda:**

```python
import boto3
import json
from datetime import datetime

# Lambda function triggered by DynamoDB Stream
def lambda_handler(event, context):
    """
    Real-time materialized view maintenance
    Updates aggregate counts as orders arrive
    """
    dynamodb = boto3.resource('dynamodb')
    aggregates_table = dynamodb.Table('real-time-aggregates')
    
    for record in event['Records']:
        if record['eventName'] in ['INSERT', 'MODIFY']:
            new_image = record['dynamodb']['NewImage']
            
            # Extract fields from DynamoDB stream format
            category = new_image['product_category']['S']
            amount = float(new_image['amount']['N'])
            timestamp = new_image['order_timestamp']['S']
            hour_key = timestamp[:13]  # YYYY-MM-DD-HH
            
            # Update real-time aggregates atomically
            try:
                aggregates_table.update_item(
                    Key={
                        'aggregate_id': f"category:{category}:hour:{hour_key}"
                    },
                    UpdateExpression="""
                        SET 
                            #cnt = if_not_exists(#cnt, :zero) + :inc,
                            total_amount = if_not_exists(total_amount, :zero) + :amt,
                            last_updated = :now
                    """,
                    ExpressionAttributeNames={
                        '#cnt': 'count'
                    },
                    ExpressionAttributeValues={
                        ':inc': 1,
                        ':amt': amount,
                        ':zero': 0,
                        ':now': datetime.utcnow().isoformat()
                    }
                )
            except Exception as e:
                print(f"Error updating aggregate: {e}")
                raise
    
    return {'processed': len(event['Records'])}
```

**Architecture Explanation:**
- **Change Data Capture (CDC):** DynamoDB Streams captures item-level modifications, enabling reactive architectures
- **Idempotent Updates:** The `if_not_exists` function ensures the update is idempotent (safe to retry)
- **Hot Partitions:** The `hour_key` creates time-bound partitions to prevent throttling on a single "category" partition key

---

## 10.4 AI/ML Integration in Data Architectures

Modern data platforms must support machine learning workflows, from feature engineering to model training and inference. Cloud providers offer managed ML platforms that integrate with data lakes and warehouses.

### 10.4.1 Feature Stores

A Feature Store is a centralized repository for storing, sharing, and serving ML features. It ensures consistency between training and inference (training-serving skew) and enables feature reuse across teams.

**Components:**
- **Offline Store:** Historical features for training (often in S3/Parquet)
- **Online Store:** Low-latency feature serving for real-time inference (Redis/DynamoDB)
- **Feature Registry:** Catalog of available features with metadata and versioning

**Amazon SageMaker Feature Store Example:**

```python
import boto3
import sagemaker
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.feature_store.inputs import TableFormatEnum

# Initialize Feature Store session
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
featurestore_runtime = boto3.client('sagemaker-featurestore-runtime')

# Define feature group for customer features
customer_fg = FeatureGroup(
    name="customer-demographics-features",
    sagemaker_session=sagemaker_session
)

# Define schema
customer_fg.load_feature_definitions(data_frame=customer_df)

# Create feature group with offline (S3) and online (DynamoDB) stores
customer_fg.create(
    s3_uri=f"s3://{bucket}/feature-store/",
    record_identifier_name="customer_id",
    event_time_feature_name="event_time",
    role_arn=role,
    enable_online_store=True,
    table_format=TableFormatEnum.ICEBERG  # Open table format for lakehouse integration
)

# Ingest features
customer_fg.ingest(data_frame=customer_df, max_workers=3, wait=True)

# Real-time feature retrieval for inference
response = featurestore_runtime.get_record(
    FeatureGroupName="customer-demographics-features",
    RecordIdentifierValueAsString="cust-12345"
)

# Features returned as key-value pairs for model input
features = {f['FeatureName']: f['ValueAsString'] for f in response['Record']}
# Result: {'age': '34', 'income_bracket': 'high', 'lifetime_value': '4500.00'}
```

### 10.4.2 MLOps Pipelines

MLOps applies DevOps principles to machine learning: version control for models, automated training pipelines, A/B testing, and model monitoring.

**SageMaker Pipelines Example:**

```python
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.processing import ScriptProcessor
from sagemaker.estimator import Estimator

# Data preprocessing step
processor = ScriptProcessor(
    image_uri=preprocess_image_uri,
    command=['python3'],
    role=role,
    instance_count=2,
    instance_type='ml.m5.xlarge'
)

step_process = ProcessingStep(
    name='PreprocessData',
    processor=processor,
    inputs=[...],
    outputs=[...],
    job_arguments=['--train-test-split-ratio', '0.2']
)

# Model training
estimator = Estimator(
    image_uri=training_image_uri,
    role=role,
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    hyperparameters={'epochs': 10}
)

step_train = TrainingStep(
    name='TrainModel',
    estimator=estimator,
    inputs={
        'training': step_process.properties.ProcessingOutputConfig.Outputs['train'].S3Output.S3Uri
    }
)

# Model evaluation
step_eval = ProcessingStep(
    name='EvaluateModel',
    processor=eval_processor,
    inputs=[...],
    property_files=[...]
)

# Conditional registration: Only register if accuracy > 0.8
cond_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name='EvaluateModel',
        property_file='evaluation',
        json_path='classification_metrics.accuracy.value'
    ),
    right=0.8
)

step_cond = ConditionStep(
    name='AccuracyCheck',
    conditions=[cond_gte],
    if_steps=[RegisterModelStep(...)],  # Register in Model Registry
    else_steps=[FailStep(...)]  # Fail pipeline if accuracy too low
)

# Create and start pipeline
pipeline = Pipeline(
    name='CustomerChurnPipeline',
    steps=[step_process, step_train, step_eval, step_cond]
)

pipeline.upsert(role_arn=role)
execution = pipeline.start()
```

### 10.4.3 Vector Databases and Retrieval-Augmented Generation (RAG)

With the rise of Large Language Models (LLMs), vector databases have become critical components of data architectures. They store embeddings (numerical representations of text, images, or audio) and enable semantic search.

**Use Case:** RAG architecture where LLMs retrieve relevant documents from a vector store before generating responses, grounding the AI in private data.

**Pinecone/OpenSearch Vector Search Example:**

```python
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import Pinecone
import pinecone

# Initialize embedding model
embeddings = BedrockEmbeddings(
    credentials_profile_name='default',
    region_name='us-east-1',
    model_id='amazon.titan-embed-text-v1'
)

# Initialize vector database
pinecone.init(api_key='key', environment='us-east-1-aws')
index = pinecone.Index('knowledge-base')

# Document ingestion pipeline
def ingest_documents(documents):
    """
    Convert documents to embeddings and store in vector DB
    Documents come from S3/Data Lake
    """
    vectorstore = Pinecone(
        index=index,
        embedding_function=embeddings.embed_query,
        text_key='text'
    )
    
    # Chunk documents and create embeddings
    vectorstore.add_texts(
        texts=[doc.page_content for doc in documents],
        metadatas=[{
            'source': doc.metadata['s3_path'],
            'category': doc.metadata['doc_type'],
            'created_at': doc.metadata['timestamp']
        } for doc in documents]
    )

# Retrieval for RAG
def retrieve_context(query, top_k=5):
    """
    Retrieve relevant documents for LLM context
    """
    vectorstore = Pinecone(
        index=index,
        embedding_function=embeddings.embed_query
    )
    
    # Semantic search finds conceptually related content, not just keyword matches
    docs = vectorstore.similarity_search(query, k=top_k)
    return "\n".join([doc.page_content for doc in docs])

# Usage in Lambda function (from Chapter 9)
def lambda_handler(event, context):
    query = event['query']
    context = retrieve_context(query)
    
    # Call LLM with retrieved context
    response = bedrock.invoke_model(
        modelId='anthropic.claude-3-sonnet',
        body=json.dumps({
            "prompt": f"Context: {context}\n\nQuestion: {query}\nAnswer:",
            "max_tokens": 500
        })
    )
    return {'answer': response}
```

---

## 10.5 Chapter Summary and Transition

This chapter has established the foundation of modern cloud data architecture, transitioning from the compute paradigms of serverless functions to the sophisticated data ecosystems that power intelligent applications. We examined the strategic differences between data lakes (flexible, schema-on-read storage for exploration) and data warehouses (structured, high-performance analytics), and explored how Lakehouse architectures unify these approaches through open table formats like Delta Lake and Apache Iceberg.

We implemented production-grade data pipelines using both ETL and ELT patterns, leveraging managed services like AWS Glue and Apache Airflow to orchestrate complex workflows without managing infrastructure. The critical shift from batch to streaming architectures was demonstrated through Apache Flink and Kinesis, enabling real-time fraud detection and operational analytics with exactly-once processing guarantees. Finally, we integrated machine learning workflows through Feature Stores, MLOps pipelines, and vector databases for AI applications.

The key insight across these patterns is the decoupling of storage from compute—whether in Redshift's separate scaling of storage and compute, BigQuery's serverless querying, or streaming architectures' independent scaling of producers and consumers. This separation enables cost optimization, elastic scaling, and technological flexibility.

However, modern organizations rarely commit to a single cloud provider for all data workloads. Regulatory requirements, best-of-breed service selection, and risk mitigation strategies increasingly demand distributing data and applications across multiple clouds and on-premises environments. The challenge shifts from optimizing within one platform to governing, securing, and moving data across heterogeneous environments.

In **Chapter 11: Hybrid and Multi-Cloud Architectures**, we will address these challenges head-on. You will learn strategies for data portability across AWS, Azure, and GCP, techniques for maintaining consistency in hybrid deployments, and governance models that provide centralized control while enabling distributed execution. We will explore Kubernetes as the common abstraction layer, data replication strategies, and the architectural patterns that allow you to leverage the best capabilities of each cloud while avoiding vendor lock-in.